In [1]:
# Arranco montando Drive para acceder a los archivos necesarios

root_path = 'drive/My Drive/Big Data/'
my_path = root_path + 'TP3/'

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
# Instalo Spark
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 22.7 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=83d2383dd3c64cba4cdbc508d309f5936ddab9a4b049600a50b784751f7c9350
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


In [3]:
from pyspark import SparkContext
sc = SparkContext("local", "MyProgram")

In [4]:
from pyspark.streaming import StreamingContext
ssc = StreamingContext(sc,15)

In [5]:
ssc.checkpoint("buffer")

In [6]:
# Realizo transformaciones
counts = ssc.textFileStream(my_path + 'input/')
mappedCounts = counts.map(lambda t: t.split("\t"))
mappedCounts = mappedCounts.map(lambda t: (t[0], t[4]))
filteredCounts = mappedCounts.filter(lambda t: t[1] != '')
mappedCounts = filteredCounts.map(lambda t: (t[0], 1))
reducedCounts = mappedCounts.reduceByKey(lambda t1, t2: t1 + t2)

def fUpdate(newValues, history):
    if(history == None):
        history = 0
    if(newValues == None):
        newValues = 0
    else:
        newValues = sum(newValues)
    return newValues + history

history = reducedCounts.updateStateByKey(fUpdate)

history.pprint()

In [ ]:
ssc.start()
ssc.awaitTermination()

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/lib/python3.8/dist-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.8/socket.py", line 669, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: ignored